# Chess.com Exporter

### Download Data from User Stats and Games Played Results and Export to CSV

----

API Docs:

* https://www.chess.com/club/chess-com-developer-community
* https://www.chess.com/news/view/published-data-api

-----

## Configure User Info

In [2]:
my_user = "markwk"

------

## Libraries and Dependencies

In [3]:
import requests
import time
import json
from datetime import datetime

import numpy as np
import pandas as pd

-----

# Get User Info and Current Stats

In [4]:
def getChessUser(username):
    r = requests.get("https://api.chess.com/pub/player/"+username+"/")
    user_info = r.json()
    return user_info

In [5]:
user_info = getChessUser(my_user)
# print(json.dumps(user_info, indent=4, sort_keys=True))

In [7]:
joined_date = datetime.utcfromtimestamp(user_info['joined'])

print(joined_date)

In [8]:
def getChessRatings(username):
    r = requests.get("https://api.chess.com/pub/player/"+username+"/stats")
    user_stats = r.json()
    return user_stats

In [169]:
user_stats = getChessRatings(my_user)
# print(json.dumps(user_stats, indent=4, sort_keys=True))

In [170]:
# latest blitz rating
print(user_stats['chess_blitz']['last'])

{'rating': 980, 'date': 1537067051, 'rd': 45}


-----

# Get Archives List

In [11]:
def getChessUserArchives(username):
    r = requests.get("https://api.chess.com/pub/player/"+username+"/games/archives")
    a = r.json()
    archives_list = a['archives']
    
    return archives_list

In [12]:
archives = getChessUserArchives(my_user)

In [13]:
# total archives / months with games
len(archives)

1

In [14]:
monthlist = []

for i in archives:
    full_archive_link = i
    month = full_archive_link.replace('https://api.chess.com/pub/player/' + my_user + '/games/',"")
    monthlist.append(month)

In [15]:
# helper function
def months(start_month, start_year, end_month, end_year):

    monthlist = []
    month, year = end_month, end_year

    while (year, month) >= (start_year, start_month):

        str = format(year) + '/' + '{:02d}'.format(month)
        monthlist.append(str)

        month -= 1
        if month < 1:
            month = 12
            year -= 1
    
    return monthlist

In [16]:
# Alternative Option 2: Get all Months Since Joined
start_year = joined_date.year
start_month = joined_date.month

end_year = datetime.now().year
end_month = datetime.now().month

In [17]:
# Alternative Option 3: manually set dates

# start_year = 2018
# start_month = 1

# end_year = 2018
# end_month = 6

In [18]:
# uncoment to run option 2 or 3
# monthlist = months(start_month, start_year, end_month, end_year)

----

# Download Data on Chess.com Games Played

In [19]:
def getChessGamesPlayed(username, monthlist):
    
    games_list = []
    
    for i in range(0,len(monthlist)):
        print("Getting Game Data for " + monthlist[i])
        
        r = requests.get("https://api.chess.com/pub/player/" + 
                         username + "/games/" + monthlist[i] +"/")
        
        r = requests.get("https://api.chess.com/pub/player/" + 
                         username + "/games/" + monthlist[i] +"/")
        
        games_obj = r.json()
        games_list.extend(games_obj['games'])
        
        # Wait for 10 seconds
        time.sleep(10)
                  
    return games_list

In [20]:
# get all games stats
games_list = getChessGamesPlayed(my_user, monthlist)

Getting Game Data for 2018/09


In [21]:
# function to unpack nested data
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

In [137]:
# unpack the nested data 
tf = pd.DataFrame(games_list)

games_played = unpack(tf, 'black', 0)
games_played = unpack(games_played, 'white', 0)

games_played.columns = ['end_time', 'fen', 'pgn', 'rated', 'rules', 'time_class',
       'time_control', 'url', 'black_id', 'black_rating', 'black_result',
       'black_username', 'white_id', 'white_rating', 'white_result', 'white_username']

In [138]:
# convert to human readable time
games_played['end_time'] = pd.to_datetime(games_played['end_time'], unit='s')

In [139]:
# set which side I played
games_played['my_side'] = np.where(games_played['white_username'] == my_user, 'white', 'black')

In [140]:
# get and assign my result
game_results = []

for index, row in games_played.iterrows():
    if row['white_username'] == my_user:
        # print("I was white")
        game_results.append(row['white_result'])
    else:
        # print("I was black")
        game_results.append(row['black_result'])
        
games_played['my_result'] = game_results

In [34]:
# export to csv
#games_played.to_csv(my_user + "_games_played_" + str(start_year) 
#                    + "-" + str(start_month) + "-to-" + str(end_year) + "-" + str(end_month) + ".csv")
games_played.to_csv("data/" + my_user + "_games_played" + ".csv")

-------

## PGN Parser: Extract Data from PGN Game Files

NOTE: To use this section, you need install the Chess-Python Library. 

You can install it via the command line with the following command: 

> pip install python-chess

In [121]:
import chess.pgn
import io
import re

In [171]:
terminations = []
total_moves = []
start_times = []
end_times  = []
moves_times_white = []
moves_times_black = []

for index, row in games_played.iterrows():
    raw_pgn = io.StringIO(row['pgn'])
    game = chess.pgn.read_game(raw_pgn)
    
    terminations.append(game.headers['Termination'] )

    start_time = game.headers['Date'] + " " + game.headers['StartTime']
    start_times.append(start_time)  

    end_time = game.headers['Date'] + " " + game.headers['EndTime']
    end_times.append(end_time) 
        
    move_times = re.findall('\{\[%clk (.*?)\]\}', row['pgn'])
    
    total_moves.append(len(move_times))
    
    # last move time: 
    # print(move_times[-1])
    
    moves_times_white.append(', '.join(move_times[0:][::2]))
    moves_times_black.append(', '.join(move_times[1:][::2]))

games_played['termination'] = terminations 

games_played['total_moves'] = total_moves                       
                       
games_played['start_time'] = start_times   
games_played['start_time'] = pd.to_datetime(games_played['start_time'])

games_played['pgn_end_time'] = end_times   
games_played['pgn_end_time'] = pd.to_datetime(games_played['pgn_end_time'])

games_played['moves_times_white'] = moves_times_white
games_played['moves_times_black'] = moves_times_black

In [172]:
# TODO: Get the play time of each player from last timestamp minus total seconds of your side game

In [166]:
games_played.head()

,end_time,fen,pgn,rated,rules,time_class,time_control,url,black_id,black_rating,...,white_result,white_username,my_side,my_result,termination,start_time,pgn_end_time,moves_times_white,moves_times_black,total_moves
0,2018-09-02 15:55:30,1r6/2kQ4/ppn1r3/1Bp5/8/b1P2N2/P4PPP/3R2K1 b - -,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",True,chess,blitz,600,https://www.chess.com/live/game/3048167960,https://api.chess.com/pub/player/dim0504,1038,...,win,markwk,white,win,markwk won by checkmate,2018-09-02 15:55:30,2018-09-02 15:55:30,"0:09:58.1, 0:09:53.4, 0:09:48, 0:09:36.4, 0:09...","0:09:56.2, 0:09:45.1, 0:09:35, 0:09:31.9, 0:09...",47
1,2018-09-03 11:06:34,r5k1/2p2ppn/2Pb4/p2BpK2/4PP1P/8/6r1/8 b - -,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",True,chess,blitz,600,https://www.chess.com/live/game/3049856809,https://api.chess.com/pub/player/markwk,1009,...,win,analardv,black,timeout,analardv won on time,2018-09-03 11:06:34,2018-09-03 11:06:34,"0:09:59.9, 0:09:56.9, 0:09:51.2, 0:09:45.5, 0:...","0:09:56.5, 0:09:48.3, 0:09:37.8, 0:09:17.7, 0:...",75
2,2018-09-03 14:01:01,rnq1k1nr/ppp2ppp/3bP3/4pP2/8/2NP4/PPP3PP/R1BQK...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",True,chess,blitz,600,https://www.chess.com/live/game/3050173823,https://api.chess.com/pub/player/mac987,933,...,win,markwk,white,win,markwk won by resignation,2018-09-03 14:01:01,2018-09-03 14:01:01,"0:09:59.3, 0:09:51.7, 0:09:45.3, 0:09:38.3, 0:...","0:09:58.1, 0:09:52.6, 0:09:48, 0:09:39.6, 0:09...",13
3,2018-09-03 14:21:52,7r/2p1kp2/p4n1p/2P1p3/1n1pP2P/5P2/4QK1P/q4B1R ...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",True,chess,blitz,600,https://www.chess.com/live/game/3050182763,https://api.chess.com/pub/player/markwk,1201,...,timeout,MoriJin99,black,win,markwk won on time,2018-09-03 14:21:52,2018-09-03 14:21:52,"0:09:59.9, 0:09:53.5, 0:09:49.9, 0:09:25, 0:09...","0:09:57.4, 0:09:50, 0:09:41.5, 0:09:33.6, 0:09...",58
4,2018-09-04 16:24:17,rn1Q2k1/ppr1nppp/8/4P3/4P3/5P2/b3K1PP/1R3B1R b...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",True,chess,blitz,600,https://www.chess.com/live/game/3052711810,https://api.chess.com/pub/player/mohitraychura,1047,...,win,markwk,white,win,markwk won by checkmate,2018-09-04 16:24:17,2018-09-04 16:24:17,"0:09:59.4, 0:09:44.7, 0:09:27.5, 0:09:21.8, 0:...","0:09:56.8, 0:09:50.1, 0:09:47.5, 0:09:19.5, 0:...",43


In [156]:
# export detailed stats to csv
games_played.to_csv("data/" + my_user + "_games_played_detailed" + ".csv")